In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import datetime
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
from keras import layers, models
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from tensorflow.keras import callbacks 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate, Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, AveragePooling2D, BatchNormalization, Dropout
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
tf.config.list_physical_devices('GPU')

[]

# Data Preparation

In [3]:
print(os.getcwd())
for x in range(3):
    os.chdir("..")
print(os.getcwd())

D:\Github\Stomach-Status-Classification\Experiment\DLApproach\OwnModel
D:\Github\Stomach-Status-Classification


In [4]:
main_data_dir = os.getcwd() + "\\Data set"
dl_data_dir = main_data_dir + "\\DL_data"
print(os.listdir(dl_data_dir))

['Abnormal', 'Normal']


In [5]:
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    validation_split = 0.2,
    horizontal_flip=True,
    fill_mode="nearest")

In [6]:
batch_size = 32
target_size = (224, 224)

In [7]:
train_generator = aug.flow_from_directory(directory = dl_data_dir, 
                                         target_size = target_size,
                                         batch_size = batch_size,
                                         subset = 'training')

val_generator = aug.flow_from_directory(directory = dl_data_dir, 
                                         target_size = target_size,
                                         batch_size = batch_size,
                                         subset = 'validation')

Found 141 images belonging to 2 classes.
Found 35 images belonging to 2 classes.


In [8]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2))

In [9]:
opt1=tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=0.1)
model.compile(optimizer=opt1,
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [10]:
Earlystop = EarlyStopping(monitor='val_loss', mode='min',patience=25, min_delta=0.001)

In [11]:
with tf.device('/CPU:0'):
    history = model.fit(train_generator,
#                         steps_per_epoch=(len(train_df)/10),
                        epochs=10,
                        validation_data=val_generator, 
#                         validation_steps=len(val_df)/batch_size,
#                         callbacks=[Earlystop]
                       )

Epoch 1/10
5/5 [==============================] - 12s 2s/step - loss: 2.2915 - accuracy: 0.6241 - val_loss: 1.1944 - val_accuracy: 0.6857
Epoch 2/10
5/5 [==============================] - 11s 2s/step - loss: 1.1311 - accuracy: 0.6809 - val_loss: 1.0672 - val_accuracy: 0.6857
Epoch 3/10
5/5 [==============================] - 11s 2s/step - loss: 1.0327 - accuracy: 0.6809 - val_loss: 1.0074 - val_accuracy: 0.6857
Epoch 4/10
5/5 [==============================] - 12s 3s/step - loss: 0.9756 - accuracy: 0.6809 - val_loss: 0.9561 - val_accuracy: 0.6857
Epoch 5/10
5/5 [==============================] - 11s 2s/step - loss: 0.9378 - accuracy: 0.6809 - val_loss: 0.9194 - val_accuracy: 0.6857
Epoch 6/10
5/5 [==============================] - 14s 3s/step - loss: 0.9059 - accuracy: 0.6809 - val_loss: 0.8915 - val_accuracy: 0.6857
Epoch 7/10
5/5 [==============================] - 11s 2s/step - loss: 0.8789 - accuracy: 0.6809 - val_loss: 0.8705 - val_accuracy: 0.6857
Epoch 8/10
5/5 [==================